In [ ]:
%pip install gymnasium
%pip install sklearn
%pip install collections
%pip install keras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 5.2 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
ERROR: Could not find a version that satisfies the requirement collections (from versions: none)
ERROR: No matching distribution found for collections


# Новый раздел

In [ ]:
import gymnasium as gym
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import RandomizedSearchCV, train_test_split
import random
from collections import deque
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from sklearn import preprocessing
from sklearn import utils


The function representing the value
of an action can be seen as a table that maps all states
and all actions to the expected long-term return. In our
case, the dimension of this table is large and compiling it
requires high computational costs.

In [ ]:
def normalize(data):
    mean = data.mean(axis=0)
    std = data.std(axis=0)
    data -= mean
    data /= std
    return data

In [ ]:
class Detector:
    def __init__(self, classifier):
        self.classifier = classifier;

    def predict(self, x):
        y = self.classifier.predict(x);
        return y

    def test(self, data):
        y = data['Malware']
        x = data.drop('Malware', axis=1)
        x = normalize(x)
        y_pred = self.classifier.predict(x)
        accuracy = accuracy_score(y, y_pred)
        matrix = confusion_matrix(y, y_pred)
        report = classification_report(y, y_pred)
        print("Accuracy:", accuracy)
        print(matrix)
        print(report)

    def fit(self, data):
        y = data['Malware']
        x = data.drop('Malware', axis=1)
        x = normalize(x)
        self.classifier.fit(x, y)

In [ ]:
class Enviroment(gym.Env):
    def __init__(self, detector, attack_traffic):
        super().__init__()
        self.result = []
        # self.result = pd.DataFrame({"Flow Duration": [], "TotLen Fwd Pkts": [], "TotLen Bwd Pkts": [],
        #                                      "Flow Byts/s": [], "Flow Pkts/s": [], "Bwd/Fwd Ratio": [], "Pkt Size Avg": [], "Malware": []})
        self.detector = detector
        self.current_state_id = 0
        self.start_states = attack_traffic
        self.start = attack_traffic.iloc[self.current_state_id]
        self.state = attack_traffic.iloc[self.current_state_id]
        self.space = [{"Name": "Flow Duration", "Action": "+Flow Duration", "Value": 0.01},
                      {"Name": "Flow Duration", "Action": "-Flow Duration", "Value": -0.01},
                      {"Name": "TotLen Fwd Pkts", "Action": "+TotLen Fwd Pkts", "Value": 0.01},
                      {"Name": "TotLen Fwd Pkts", "Action": "-TotLen Fwd Pkts", "Value": -0.01},
                      {"Name": "TotLen Bwd Pkts", "Action": "+TotLen Bwd Pkts", "Value": 0.01},
                      {"Name": "TotLen Bwd Pkts", "Action": "-TotLen Bwd Pkts", "Value": -0.01},
                      {"Name": "Flow Byts/s", "Action": "+Flow Byts/s", "Value": 0.01},
                      {"Name": "Flow Byts/s", "Action": "-Flow Byts/s", "Value": -0.01},
                      {"Name": "Flow Pkts/s", "Action": "+Flow Pkts/s", "Value": 0.01},
                      {"Name": "Flow Pkts/s", "Action": "-Flow Pkts/s", "Value": -0.01},
                      {"Name": "Bwd/Fwd Ratio", "Action": "+Bwd/Fwd Ratio", "Value": 0.01},
                      {"Name": "Bwd/Fwd Ratio", "Action": "-Bwd/Fwd Ratio", "Value": -0.01},
                      {"Name": "Pkt Size Avg", "Action": "+Pkt Size Avg", "Value": 0.01},
                      {"Name": "Pkt Size Avg", "Action": "-Pkt Size Avg", "Value": -0.01}]

    def reset(self):
        self.state = self.start
        self.current_state_id = 0
        return self.state.values.reshape(1, -1)

    def step(self, action_n):
        action = self.space[action_n]
        self.state[action["Name"]] += action["Value"]
        result = self.detector.predict(self.state.values.reshape(1, -1))
        if result == 0:
          curr_state = self.state.copy()
          curr_state["Malware"] = 1
          self.result.append(curr_state.to_dict())

          # self.result = pd.concat([self.result, curr_state], ignore_index=True)
          self.current_state_id = self.current_state_id + 1 if self.current_state_id + 1 < len(self.start_states) else 0
          self.state = self.start_states.iloc[self.current_state_id]
        return (self.state.values.reshape(1, -1), 1 if result == 0 else 0, False)

In [ ]:
class DQLAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size # Сколько фич изменяем
        self.action_size = action_size # В каком объеме меняем
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_decay = 0.995
        self.epsilon_min = 0.01
        self.model = self.build_model()

    def build_model(self):
        model = Sequential()
        model.add(Dense(512, input_dim=self.state_size, activation='relu'))
        model.add(Dense(256, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        opt = keras.optimizers.Adam(use_ema=True)
        model.compile(loss='mse', optimizer=opt)
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])

    def replay(self, batch_size):
        batch_size = min(batch_size, len(self.memory))
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = (reward + self.gamma * np.amax(self.model.predict(next_state)[0]))
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

In [ ]:
class Learner:
  def __init__(self, agent, env):
    self.agent = agent
    self.env = env

  def fit(self, epoch_n):
    state = self.env.reset()
    for _ in range(epoch_n):
      action = self.agent.act(state)
      new_state, reward, done = self.env.step(action)
      self.agent.remember(state, action, reward, new_state, done)
      self.agent.replay(10)

In [ ]:
data = pd.read_csv('Botnet-Training-Short.csv')
forest = RandomForestClassifier()
detector = Detector(forest)
detector.fit(data)
test_data = pd.read_csv('Botnet-Testing-Short.csv')
detector.test(test_data)

Accuracy: 0.7460011946638349
[[21633  1410]
 [ 6244   847]]
              precision    recall  f1-score   support

           0       0.78      0.94      0.85     23043
           1       0.38      0.12      0.18      7091

    accuracy                           0.75     30134
   macro avg       0.58      0.53      0.52     30134
weighted avg       0.68      0.75      0.69     30134



In [ ]:
df = pd.read_csv('Botnet-Training-Short.csv')
state = df.loc[df['Malware'] == 1]
state = state.drop('Malware', axis=1)
state = normalize(state)
state = state.head(1)
env = Enviroment(detector, state)
agent = DQLAgent(7,14)
learner = Learner(agent, env)
learner.fit(10)



1/1 [==============================] - 0s 19ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


1/1 [==============================] - 0s 17ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


1/1 [==============================] - 0s 16ms/step


...

1/1 [==============================] - 0s 16ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


1/1 [==============================] - 0s 19ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


1/1 [==============================] - 0s 17ms/step


In [ ]:
data_new = pd.read_csv('Botnet-Training-Short.csv')
forest_new = RandomForestClassifier()
detector_new = Detector(forest_new)
data_new = pd.concat([data_new, pd.DataFrame(learner.env.result)], ignore_index=True)
detector_new.fit(data_new)
test_data = pd.read_csv('Botnet-Testing-Short.csv')
detector.test(test_data)

Accuracy: 0.7460011946638349
[[21633  1410]
 [ 6244   847]]
              precision    recall  f1-score   support

           0       0.78      0.94      0.85     23043
           1       0.38      0.12      0.18      7091

    accuracy                           0.75     30134
   macro avg       0.58      0.53      0.52     30134
weighted avg       0.68      0.75      0.69     30134



        self.space = [{"Name": "Flow Duration", "Action": "+Flow Duration", "Value": 0.01},
                      {"Name": "Flow Duration", "Action": "-Flow Duration", "Value": -0.01},
                      {"Name": "TotLen Fwd Pkts", "Action": "+TotLen Fwd Pkts", "Value": 0.01},
                      {"Name": "TotLen Fwd Pkts", "Action": "-TotLen Fwd Pkts", "Value": -0.01},
                      {"Name": "TotLen Bwd Pkts", "Action": "+TotLen Bwd Pkts", "Value": 0.01},
                      {"Name": "TotLen Bwd Pkts", "Action": "-TotLen Bwd Pkts", "Value": -0.01},
                      {"Name": "Flow Byts/s", "Action": "+Flow Byts/s", "Value": 0.01},
                      {"Name": "Flow Byts/s", "Action": "-Flow Byts/s", "Value": -0.01},
                      {"Name": "Flow Pkts/s", "Action": "+Flow Pkts/s", "Value": 0.01},
                      {"Name": "Flow Pkts/s", "Action": "-Flow Pkts/s", "Value": -0.01},
                      {"Name": "Bwd/Fwd Ratio", "Action": "+Bwd/Fwd Ratio", "Value": 0.01},
                      {"Name": "Bwd/Fwd Ratio", "Action": "-Bwd/Fwd Ratio", "Value": -0.01},
                      {"Name": "Pkt Size Avg", "Action": "+Pkt Size Avg", "Value": 0.01},
                      {"Name": "Pkt Size Avg", "Action": "-Pkt Size Avg", "Value": -0.01}]

In [ ]:
# class CEM():
#     def __init__(self, state_n, action_n):
#       self.state_n = state_n
#       self.action_n = action_n
#       self.policy = np.ones((self.state_n, self.action_n)) / self.action_n

#     def get_action(self, state):
#       return int(np.random.choice(np.arange(self.action_n), p=self.policy[state]))

#     def update_policy(self, elite_tr):
#       pre_policy = np.zeros((state_n, action_n))

#       for tr in elite_tr:
#         for state, action in zip(tr['states'], tr['actions']):
#           pre_policy[state][action] += 1

#           for state in range(self.action_n):
#             if sum(pre_policy[state]) == 0:
#               self.policy[state] = np.ones(self.action_n) / self.action_n
#             else:
#               self.policy[state] = pre_policy[state] / sum(pre_policy[state])

In [ ]:
# class CEMAgent():
#     def __init__(self, env, agent_logic):
#       self.env = env
#       self.agent_logic = agent_logic

#     def get_state(self):
#       return None

#     def get_trajectory(self, tr_len):
#       tr = {'states': [], 'actions': [], 'total_reward': 0}

#       obs = self.env.reset()
#       state = get_state(obs)
#       tr['states'].append(state)

#       for _ in range(tr_len):
#           action = self.agent_logic.get_action(state)
#           tr['actions'].append(action)

#           obs, reward, done = env.step(action)
#           state = get_state(obs)
#           tr['total_reward'] += reward

#           if done:
#               break

#           tr['states'].append(state)

#         return tr
#       def get_elite_trajectories(self, trajectories, q):
#           total = [tr['total_reward'] for tr in trajectories]
#           quantile = np.quantile(total, q=q)
#           return [tr for tr in trajectories if tr['total_reward'] > quantile]

#       def fit(self, episode_n, trajectory_n, trajectory_len, q):
#           for _ in range(episode_n):
#               trajectories = [get_trajectory(trajectory_len) for _ in range(trajectory_n)]

#               mean_total = np.mean([trajectory['total_reward'] for trajectory in trajectories])
#               print(mean_total)
#               elite = get_elite_trajectories(trajectories, q)

#               if len(elite) > 0:
#                   self.agent_logic.update_policy(elite)